In [17]:
import os
import sys
import subprocess
import signac
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statistics import linear_regression
from scipy.stats import linregress
from pymser import pymser

In [24]:
def plot_res_pymser(job, eq_col, results, name, box_name, eq_fig = False):
    fig, [ax1, ax2] = plt.subplots(1, 2, gridspec_kw={'width_ratios': [2, 1]}, sharey=True)

    ax1.set_ylabel(name, color="black", fontsize=14, fontweight='bold')
    ax1.set_xlabel("GEMC step", fontsize=14, fontweight='bold')

    ax1.plot(range(len(eq_col)), 
            eq_col, 
            label = 'Raw data', 
            color='blue')

    ax1.plot(range(len(eq_col))[results['t0']:], 
            results['equilibrated'], 
            label = 'Equilibrated data', 
            color='red')

    ax1.plot([0, len(eq_col)], 
            [results['average'], results['average']], 
            color='green', zorder=4, 
            label='Equilibrated average')

    ax1.fill_between(range(len(eq_col)), 
                    results['average'] - results['uncertainty'], 
                    results['average'] + results['uncertainty'], 
                    color='lightgreen', alpha=0.3, zorder=4)

    ax1.set_yticks(np.arange(0, eq_col.max()*1.1, eq_col.max()/10))
    ax1.set_xlim(-len(eq_col)*0.02, len(eq_col)*1.02)
    ax1.tick_params(axis="y", labelcolor="black")

    ax1.grid(alpha=0.3)
    ax1.legend()

    ax2.hist(eq_col, 
            orientation=u'horizontal', 
            bins=30, 
            edgecolor='blue', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ax2.hist(results['equilibrated'], 
            orientation=u'horizontal', 
            bins=3, 
            edgecolor='red', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ymax = int(ax2.get_xlim()[-1])

    ax2.plot([0, ymax], 
            [results['average'], results['average']],
            color='green', zorder=4, label='Equilibrated average')

    ax2.fill_between(range(ymax), 
                    results['average'] - results['uncertainty'],
                    results['average'] + results['uncertainty'],
                    color='lightgreen', alpha=0.3, zorder=4)

    ax2.set_xlim(0, ymax)

    ax2.grid(alpha=0.5, zorder=1)

    fig.set_size_inches(9,5)
    fig.set_dpi(100)
    fig.tight_layout()
    save_name = 'MSER_eq_'+ box_name +'.png'
#     plt.show()
    print("Success! Saving figure")
#     fig.savefig(job.fn(save_name), dpi=300, facecolor='white')
    plt.close(fig)

In [25]:
def check_equil_converge(job, eq_data_dict, prod_tol):
    equil_matrix = []
    res_matrix = []
    prop_cols = [5]
    prop_names = ["Number of Moles"]
    try:
        # Load data for both boxes
        for key in list(eq_data_dict.keys()):
            eq_col = eq_data_dict[key]["data"]
        # df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))
        # df_box2 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))

        # Process both boxes in one loop
        # for box in [df_box1, df_box2]:
            # for prop_index in prop_cols:
            #     eq_col = box[:, prop_index - 1]
            # print(len(eq_col))
            batch_size = max(1, int(len(eq_col) * 0.0005))

            # Try with ADF test enabled, fallback without it if it fails
            try:
                results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=True, uncertainty='uSD', print_results=False)
                adf_test_failed = results["critical_values"]["1%"] <= results["adf"]
            except:
                results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=False, uncertainty='uSD', print_results=False)
                results["adf"], results["critical_values"], adf_test_failed = None, None, False

            equilibrium = len(eq_col) - results['t0'] >= prod_tol
            equil_matrix.append(equilibrium and not adf_test_failed)
            res_matrix.append(results)

        # Log results
        # print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
        # print(equil_matrix)
        # log_text = '==============================================================================\n'
        
        for i, is_equilibrated in enumerate(equil_matrix):
            # box = df_box1 if i < len(prop_cols) else df_box2
            # box_name = "Liquid" if i < len(prop_cols) else "Vapor"
            # col_vals = box[:, prop_cols[i % len(prop_cols)] - 1]
            key_name = list(eq_data_dict.keys())[i]
            box_name = key_name.rsplit("_", 1)[0]
            col_vals = eq_data_dict[key_name]["data"]
            #plot all

            if all(equil_matrix):
                plot_res_pymser(job, col_vals, res_matrix[i], prop_names[i % len(prop_cols)], box_name)

            # Display outcome
            prod_cycles = len(col_vals) - res_matrix[i]['t0']
            if is_equilibrated:
                #Plot successful equilibration
                statement = f"       > Success! Found {prod_cycles} production cycles."
            else:
                #Plot failed equilibration
                statement = f"       > {box_name} Box Failure! "
                if res_matrix[i]["adf"] is None:
                    # Note: ADF test failed to complete
                    statement += f"ADF test failed to complete! "
                elif res_matrix[i]['adf'] > res_matrix[i]['critical_values']['1%']:
                    adf, one_pct = res_matrix[i]['adf'], res_matrix[i]['critical_values']['1%']
                    statement += f"ADF value: {adf}, 99% confidence value: {one_pct}! "
                if len(col_vals) - res_matrix[i]['t0'] < prod_tol:
                    statement += f"Only {prod_cycles} production cycles found."
                
            # print(statement)

    except Exception as e:
        #This will cause an error in the GEMC operation which lets us know that the job failed
        raise Exception(f"Error processing job {job.id}: {e}")

    return all(equil_matrix) 

In [26]:
def run_gemc(job):
    "Run gemc"
    # Move into the job dir and start doing things
    use_crit = False
    try:
        #Inititalize counter and number of eq_steps
        count = 1
        total_eq_steps = job.sp.nsteps_eq
        prod_tol = int(job.sp.nsteps_eq/4)
        #Originally set the document eq_steps to 1 larger than the max number, it will be overwritten later
        total_steps = int(job.sp.nsteps_eq*4+1)
        with job:

            prop_cols = [5] #Use number of moles to decide equilibrium
            # Load initial eq data from both boxes
            df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))
            df_box2 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))

            # Process both boxes in one loop
            eq_data_dict = {}
            for b, box in enumerate([df_box1, df_box2]):
                box_name = "Liquid" if b == 0 else "Vapor"
                for prop_index in prop_cols:
                    eq_col = box[:, prop_index - 1]
                    #Save eq_col as a csv for later analysis
                    key = f"{box_name}_{prop_index}"
                    eq_col_file = job.fn(f"{box_name}_eq_col_{prop_index}.csv")
                    # np.savetxt(eq_col_file, eq_col, delimiter=",")
                    #Save the eq_col and file to a dictionary for later use
                    eq_data_dict[key] = {"data": eq_col, "file": eq_col_file}

            prod_tol_eq = int(eq_data_dict[key]["data"].size/4)
            # print(f"Prod Tol: {prod_tol_eq}")
            #While we are using at most 12 attempts to equilibrate
            while count <= 13:
                # Check if equilibration is reached via the pymser algorithms
                is_equil = check_equil_converge(job, eq_data_dict, prod_tol_eq)
                #If equilibration is reached, break the loop and start production
                if is_equil:
                    break
                else:
                    #Increase the total number of eq steps by 25% of the original value and restart the simulation
                    total_eq_steps += int(prod_tol)
                    #If we've exceeded the maximum number of equilibrium steps, raise an exception
                    #This forces a retry with critical conditions or will note complete GEMC failure
                    if count == 13:
                        # job.doc.equil_fail = True
                        use_crit = True
                        raise Exception(f"GEMC equilibration failed to converge after {job.sp.nsteps_eq*4} steps")
                    #Otherwise continue equilibration
                    else:
                        #Add restart data to eq_col
                        # After each restart, load the updated properties data for both boxes
                        #Use fake data to extend simulation
                        df_box1r = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))[int(3*len(df_box1)/4):]
                        df_box2r = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))[int(3*len(df_box2)/4):]


                        # Process and add the restart data to eq_col for each property in each box
                        for b, box in enumerate([df_box1r, df_box2r]):
                            box_name = "Liquid" if b == 0 else "Vapor"
                            for i, prop_index in enumerate(prop_cols):
                                #Get the key from the property and box name
                                key = f"{box_name}_{prop_index}"
                                # Extract the column data for this restart and append to accumulated data
                                eq_col_restart = box[:, prop_index - 1]
                                eq_col_data = eq_data_dict[key]["data"]
                                all_eq_data = np.concatenate((eq_col_data, eq_col_restart))
                                #Save the new data to the eq_col file
                                # print(eq_data_dict[key]["file"])
                                # np.savetxt(eq_data_dict[key]["file"], all_eq_data, delimiter=",")
                                #Overwite the current data in the eq_data_dict with restart data
                                # print("orig len", len(eq_data_dict[key]["data"]))
                                # print(eq_data_dict[key]["data"][0:5])
                                eq_data_dict[key]["data"] = all_eq_data
                                # print("totla len", len(eq_data_dict[key]["data"]))
                #Increase the counter
                count += 1

            #Set the step counter to whatever the final number of equilibration steps was
            total_steps = total_eq_steps

            # Run production
            print("Running production")
    except:
        # if GEMC failed with critical conditions as intial conditions, terminate with error
        if use_crit == True:
            # If so, terminate with error and log failure in job document
            # job.doc.gemc_failed = True
            print("GEMC failed with critical and experimental starting conditions and the molecule is " + job.sp.mol_name + " at temperature " + str(job.sp.T))
            # )
        else:
            print("Reatart w/ Critical Conditions")
            # Otherwise, try with critical conditions


In [28]:
import glob
def process_old_job(jobs):
    "Run gemc"
    for job in jobs:
        print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
        prop_cols = [5] #Use number of moles to decide equilibrium
        # Load initial eq data from both boxes
        try:
            df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))
            df_box2 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
            
            # Process both boxes in one loop
            eq_data_dict = {}
            for b, box in enumerate([df_box1, df_box2]):
                box_name = "Liquid" if b == 0 else "Vapor"
                for prop_index in prop_cols:
                    eq_col = box[:, prop_index - 1]
                    #Save eq_col as a csv for later analysis
                    key = f"{box_name}_{prop_index}"
                    eq_col_file = job.fn(f"{box_name}_eq_col_{prop_index}.csv")
                    # np.savetxt(eq_col_file, eq_col, delimiter=",")
                    #Save the eq_col and file to a dictionary for later use
                    eq_data_dict[key] = {"data": eq_col, "file": eq_col_file}
            prod_tol_eq = int(eq_data_dict[key]["data"].size/4) 
            is_equilibrated = check_equil_converge(job, eq_data_dict, prod_tol_eq)
            #If all jobs pass, set nsteps_eq needed to the original amount
            if is_equilibrated:
                print("Equilibrated setting job doc nsteps")
                # job.doc.nsteps_eq = job.sp.nsteps_eq
            #Otherwise, delete job production information, equil info will be retained and used since job.doc.nsteps_eq is not set 
            else:
                with job:
                    print("Deleting production data")
                    # for file_path in glob.glob(os.path.join(job.fn(""), "prod.*")):
                    #     # print("Deleting", file_path)
                    #     os.remove(file_path)
                        #pass
        except:
            print("Error processing job", job.id)


In [ ]:
import glob
mol_name = "R170"
project = signac.get_project("opt_ff_ms")
# jobs = project.find_jobs({"mol_name": mol_name, "T": 250, "atom_type": 8})
jobs = project.find_jobs()
process_old_job(jobs)

ID a0ed113a5220ff7fbf9b6842bf619be0 AT 8 T 190.0
The results may not be reliable!
The results may not be reliable!
ID 6669d1a1d1e8912231aea26c8a144c94 AT 8 T 210.0
The results may not be reliable!
The results may not be reliable!
ID 9a6a7e7e4e9c2e964b45bed7f3149bf8 AT 8 T 230.0
The results may not be reliable!
The results may not be reliable!
ID 924ac2fcc7459a70f9d59d9414aa43aa AT 8 T 250.0
Error processing job 924ac2fcc7459a70f9d59d9414aa43aa
ID 44338ea247e05616d310bece4fac69d0 AT 8 T 270.0
The results may not be reliable!
The results may not be reliable!
The results may not be reliable!
The results may not be reliable!
ID 25e9e57de24e6ab9b5a0b2ee74c84bad AT 8 T 260.0
The results may not be reliable!
The results may not be reliable!
ID 4b0f344e45061afc7886ab256695142b AT 8 T 280.0
The results may not be reliable!


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)


The results may not be reliable!
ID a9e80c4a08f675d53b93388c99cf3a52 AT 8 T 300.0
The results may not be reliable!
The results may not be reliable!
ID 1fdc2542cb82f7904e8ebf7074032247 AT 8 T 320.0
The results may not be reliable!
The results may not be reliable!
ID 70a9341c713d34ec24c56e02e9aa618d AT 8 T 340.0
The results may not be reliable!
The results may not be reliable!
ID cec044435ac5ad878915e4ab471cf010 AT 8 T 260.0
ID fc67389850f0afb944d92c498d7c0d91 AT 8 T 280.0
The results may not be reliable!
The results may not be reliable!
ID 57a003793cdd64f6b86d4472fe31f7da AT 8 T 300.0
The results may not be reliable!
The results may not be reliable!
ID 9303e459421d0147a5e940a9404caecc AT 8 T 320.0
The results may not be reliable!


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site

The results may not be reliable!
ID 0247d53a5e58f5d13dacc01f12073c7b AT 8 T 340.0
The results may not be reliable!
The results may not be reliable!
ID 510c0ad2b2060353933c9f22586a407c AT 8 T 240.0
The least-squares minimization failed! Please check the data.
`ydata` must not be empty!
The least-squares minimization failed! Please check the data.
`ydata` must not be empty!


...

ID 67f4967dea9638bb97302283c949fa3d AT 2 T 210.0
The results may not be reliable!
The results may not be reliable!
The results may not be reliable!
The results may not be reliable!
ID a503754acec61b1f059c99086f50d102 AT 2 T 241.0


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)


ID ac77acb9dbd46c322dc4d7e053dbc9cc AT 1 T 250.0
Success! Saving figure
Success! Saving figure
Equilibrated setting job doc nsteps
ID 2005f64b0197a8f5c49f2d395329ed79 AT 1 T 270.0
Success! Saving figure
Success! Saving figure
Equilibrated setting job doc nsteps
ID 1cb1db42a456ce7c919b2af3a003cff9 AT 1 T 290.0
ID c2387535cb5caeb9def852e7b59659ee AT 1 T 210.0
ID 6afdd9c2fcf3d73aadb213da38f7eeea AT 1 T 230.0
The results may not be reliable!
The results may not be reliable!
ID 945c5987f71c30cbbee3b7b95b335a21 AT 1 T 250.0
Success! Saving figure
Success! Saving figure
Equilibrated setting job doc nsteps
ID ede604ae90fd19e8c8a781593e779f85 AT 1 T 270.0
ID 38ad1aa8eb2011f713a9a22a09da9f3b AT 1 T 290.0
ID f5547cf93e8e7eafd2fe1c87b01e4c37 AT 1 T 190.0
The results may not be reliable!
The results may not be reliable!
ID f587cf9ddcac4454c753e7030ea2761a AT 1 T 210.0
ID 96e813213e900efb44fde0bba07af2da AT 1 T 230.0
The results may not be reliable!
The results may not be reliable!
ID 554f3d75923cb8

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/pymser/pymser.py:439: OptimizeWarning: Covariance of the parameters could not be estimated
  [tau], _ = curve_fit(exp_decay,  x,  ACF)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


Success! Saving figure
Success! Saving figure
Equilibrated setting job doc nsteps
ID 02061a6e064dc3b852199f9824b59fbf AT 1 T 270.0
Success! Saving figure


In [ ]:
import glob
mol_name = "R170"
project = signac.get_project("opt_ff_ms")
jobs = project.find_jobs({"mol_name": mol_name, "T": 250, "atom_type": 8})
jobs = project.find_jobs({"mol_name": mol_name})
# jobs = list(project.find_jobs({"mol_name": mol_name, "T": 290, "atom_type": 6}))[0]
# jobs = list(project.find_jobs({"mol_name": mol_name, "T": 230, "atom_type": 1}))[0]
# print(job)
for job in jobs:
    # print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
    run_gemc(job)